# Numpy

* Пакет для Python, реализующий многомерные массивы
* Массивы **статически типизированные**. Тип элементов определяется при создании массива.
* Массивы эффективно используют память.
* Методы линейной алгебры в numpy реализованы на C и Fortran, что обеспечивает хорошую производительность.
* Пакет numpy активно используется в научных проектах по всему миру.

In [2]:
import numpy as np

## Создание массивов

Существует несколько способов создания новых numpy массивов:
* через списки и кортежи Python
* используя специальные функции numpy такие, как `arange`, `linspace`, и так далее.
* вычитывая данные из файлов

### 1D массивы:

In [3]:
a = np.array([0,1,2,3])
a

array([0, 1, 2, 3])

In [4]:
type(a), a.dtype

(numpy.ndarray, dtype('int32'))

In [5]:
a.ndim, a.shape, len(a)

(1, (4,), 4)

In [ ]:
b = np.array((3, 4, 5))
b

В numpy существует множество функций для генерации массивов:

#### `arange`

Генерирует значения в интервале [start, stop) с шагом step. Аналог встроенной функции Python `range`. На уровне типов они, конечно, отличаются. `np.arange` возвращает обычный numpy-массив, в то время как `range` является lazy sequence/lazy iterable и стоит в общем ряду с `list` и `tuple`. Funny fact: `range` не является итератором -- он не "иссякает", для него нельзя вызвать `next()` и можно `len()`, но, в отличие от списка, он lazy

In [114]:
x = np.arange(0, 10, 1) # аргументы: start, stop, step
type(range(10))


range

In [8]:
x = np.arange(-1, 1, 0.1)
x

array([-1.00000000e+00, -9.00000000e-01, -8.00000000e-01, -7.00000000e-01,
       -6.00000000e-01, -5.00000000e-01, -4.00000000e-01, -3.00000000e-01,
       -2.00000000e-01, -1.00000000e-01, -2.22044605e-16,  1.00000000e-01,
        2.00000000e-01,  3.00000000e-01,  4.00000000e-01,  5.00000000e-01,
        6.00000000e-01,  7.00000000e-01,  8.00000000e-01,  9.00000000e-01])

#### `linspace` и `logspace`

`linspace` Генерирует равномерно распределенные числа, включая конечные точки.

`logspace` То же, но в логарифмической шкале.

In [10]:
np.linspace(0, 10, 10) # аргументы: start, stop, число точек

array([ 0.        ,  1.11111111,  2.22222222,  3.33333333,  4.44444444,
        5.55555556,  6.66666667,  7.77777778,  8.88888889, 10.        ])

In [ ]:
np.logspace(0.1, 1, 4, base=2)

#### `zeros`, `ones`, `zeros_like` и `ones_like`

In [12]:
np.zeros((5,))   # Аргумент должен быть кортежем

array([0., 0., 0., 0., 0.])

In [13]:
a = np.ones((4,))
a

array([1., 1., 1., 1.])

In [14]:
b = np.zeros_like(a)
b

array([0., 0., 0., 0.])

In [15]:
c = np.ones_like(b)
c

array([1., 1., 1., 1.])

### Многомерные массивы

In [16]:
# Матрица
m = np.array([[1., 2.], [3., 4.]])
m


array([[1., 2.],
       [3., 4.]])

In [123]:
type(m), m.dtype

(numpy.ndarray, dtype('int32'))

In [18]:
m.ndim, np.shape(m), len(m), np.size(m)

(2, (2, 2), 2, 4)

При попытке назначить значение другого типа будет выдана ошибка:

In [ ]:
m[0,0] = 'hello' 

#### `zeros`, `ones`, `zeros_like` и `ones_like`

In [19]:
a = np.ones((3, 3))
a

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [ ]:
b = np.zeros((2, 2))
b

#### Другие функции

In [20]:
c = np.eye(3) # единичная матрица
c

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [21]:
d = np.diag([3, 5, 7]) # диагональная матрица
d

array([[3, 0, 0],
       [0, 5, 0],
       [0, 0, 7]])

### Упражнение 1
Построить диагональную матрицу, на диагонали которой расположены числа от 0 до 3.

In [25]:
np.diag(np.arange(0,3+1,1))

array([[0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 2, 0],
       [0, 0, 0, 3]])

### Упражнение 2

Создать матрицу размерности 3x3x3 со случайными значениями, имеющими равномерное распределение от 1 до 5.

In [38]:
np.random.uniform(low=1, high=5, size=(3,3,3))


array([[[3.08551982, 2.35177202, 2.66450646],
        [4.64624387, 4.31198672, 2.09757192],
        [2.21049638, 4.28616149, 1.2437726 ]],

       [[2.99975625, 1.13830107, 4.21343904],
        [4.52419469, 1.79529933, 1.26486473],
        [4.06580531, 2.5012074 , 4.61567954]],

       [[1.17990014, 1.66307673, 4.80286122],
        [4.1424824 , 4.35270341, 3.61801168],
        [1.37812172, 2.82019619, 1.52666305]]])

## Копирование в NumPy

Как мы помним, в Python при присваивании не происходит копирование объектов. 

In [39]:
m = np.array([[1, 2], [3, 4]])
m

array([[1, 2],
       [3, 4]])

In [40]:
n = m

In [41]:
# Изменение N меняет M
n[0, 0] = 10
n

array([[10,  2],
       [ 3,  4]])

In [ ]:
m

Глубокая копия создается в NumPy с помощью функции `copy`:

In [ ]:
n = np.copy(m)

In [ ]:
# теперь при изменении N M остается нетронутым
n[0,0] = -5
n

In [ ]:
m

Слайсинг в numpy создает лишь представление изначального массива, т.е. копирования в памяти не происходит.

При изменении представления меняется и изначальный массив:

In [ ]:
a = np.arange(10)
a

In [ ]:
b = a[::2]
b

In [ ]:
b[0] = 12
b

In [ ]:
a # (!!)

In [ ]:
a = np.arange(10)
b = a[::2].copy() # глубокое копирование
b[0] = 12
a

## Слияние массивов

Функции `vstack`, `hstack` и `concatenate` позволяются составить общий массив из нескольких массивов:

### `concatenate`

In [43]:
a = np.array([[1, 2], [3, 4]])
a

array([[1, 2],
       [3, 4]])

In [44]:
b = np.array([[5, 6]])
b

array([[5, 6]])

In [45]:
np.concatenate((a, b), axis=0)

array([[1, 2],
       [3, 4],
       [5, 6]])

In [48]:
np.concatenate((a, b.T), axis=1)

array([[1, 2, 5],
       [3, 4, 6]])

### `hstack` и `vstack`

In [49]:
np.vstack((a,b))

array([[1, 2],
       [3, 4],
       [5, 6]])

In [50]:
np.hstack((a,b.T))

array([[1, 2, 5],
       [3, 4, 6]])

## Индексация

Доступ к данным массива организуется с помощью индексов и оператора `[]`.

In [ ]:
a = np.arange(10)
a

In [ ]:
a[0], a[2], a[-1]

Для многомерных массивов индексами является кортеж целых чисел:

In [ ]:
m = np.diag(np.arange(3))
m

In [ ]:
m[1, 1]

In [ ]:
m[(1, 1)]

Можно использовать "`:`" для получения доступа к целой колонке или строке: 

In [ ]:
m[1, :] # строка 1

In [ ]:
m[:, 2] # колонка 2

Присваивание новых значений элементам массива:

In [ ]:
m[2, 1] = 10
m

In [ ]:
m[1, :] = 5
m[: ,2] = -1

In [ ]:
m

## Слайсинг

NumPy поддерживает слайсинг, как и списки с кортежами в Python:

In [58]:
a = np.arange(10)
a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

Все три параметра слайсинга являются опциональными: по умолчанию `start` равен **0**, `end` равен последнему элемену и `step` равен **1** в `a[start:stop:step]`:

In [59]:
a[::] # все параметры слайсинга имеют значения по умолчанию

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [60]:
a[1:3]

array([1, 2])

In [ ]:
a[:3]

In [ ]:
a[3:]

In [ ]:
a[2:9:2] 

Отрицательные индексы отсчитываются от конца массива:

In [ ]:
a[-1] # последний элемент массива

In [ ]:
a[-3:] # последние три элемента

Слайсы являются представлениями массива, а потому являются изменяемыми:

In [ ]:
a[1:3] = [-2,-3]
a

Слайсинг работает точно так же и для многомерных массивов:

In [61]:
m = np.random.randint(1,100, size=(4, 4))
m

array([[46,  8, 51,  4],
       [57, 37, 39, 72],
       [27, 29, 98, 75],
       [86, 91, 11, 33]])

In [62]:
m[1:4, 1:4]

array([[37, 39, 72],
       [29, 98, 75],
       [91, 11, 33]])

In [ ]:
m[::2, ::2]

### Упражнение 3
Создать нулевую матрицу размерности 5х5 с единицами по ее "границам".

In [68]:
a = np.zeros([5,5])
a[0] = 1
a[4] = 1
a[1:5, 0] = 1
a[1:5, 4] = 1
a

array([[1., 1., 1., 1., 1.],
       [1., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1.],
       [1., 1., 1., 1., 1.]])

## Линейная алгебра

Код, написанный на numpy становится эффективным тогда, когда он достаточно векторизован (т.е. векторные и матричные операции используются в бОльшей части программы).

### Поэлементные операции

Все арифметические операции по умолчанию являются поэлементными:

In [69]:
a = np.arange(4)
a + 1

array([1, 2, 3, 4])

In [70]:
5*a

array([ 0,  5, 10, 15])

In [71]:
2**a

array([1, 2, 4, 8], dtype=int32)

In [72]:
j = np.arange(5)
2**(j + 1) - j

array([ 2,  3,  6, 13, 28])

### Операции между массивами

In [73]:
a = np.arange(4)
b = np.ones(4) + 3
print('a = ', a)
print('b = ', b)
a - b

a =  [0 1 2 3]
b =  [4. 4. 4. 4.]


array([-4., -3., -2., -1.])

In [74]:
a * b

array([ 0.,  4.,  8., 12.])

Сравнения:

In [75]:
a = np.array([1, 2, 3, 4])
b = np.array([4, 2, 2, 4])
a == b

array([False,  True, False,  True])

In [76]:
a > b

array([False, False,  True, False])

Логические операции:

In [77]:
a = np.array([1, 1, 0, 0], dtype=bool)
b = np.array([1, 0, 1, 0], dtype=bool)
np.logical_or(a, b)

array([ True,  True,  True, False])

In [78]:
a | b

array([ True,  True,  True, False])

In [79]:
np.logical_and(a, b)

array([ True, False, False, False])

In [80]:
a & b

array([ True, False, False, False])

Использование логических операторов позволяет обращаться к элементам массива через маску:

In [81]:
c = np.array([1, 2, 3, 4], dtype=int)
d = np.array([1, 2, 5, 6], dtype=int)
c[c == d]

array([1, 2])

### Упражнение 4
Создать случайный вектор размера 10 и найти его среднее значение.

In [130]:
a = np.random.random((10,))
print(a.mean())
print(a.sum() / len(a))
a


0.4525140991010031
0.4525140991010031


array([[0.72170027, 0.27754879],
       [0.03758802, 0.45928636],
       [0.47261656, 0.73116237],
       [0.99750598, 0.74044374],
       [0.66502089, 0.61348561],
       [0.44917311, 0.26874203],
       [0.11518284, 0.57393679],
       [0.30768825, 0.66808819],
       [0.12442999, 0.87424624],
       [0.31239193, 0.83929288]])

### Упражнение 5
Создать случайную матрицу размерности 10х2, хранящую декартовы координаты. Затем конвертируйте их в полярные координаты.

In [143]:

decart_c = np.random.randint(0, 100, size=(10, 2))
print("Декартовы координаты:")
print(decart_c)
x = decart_c[:, 0]
y = decart_c[:, 1]
r = np.sqrt(x**2 + y**2)
fi = np.arctan2(y, x)
polar_c = np.column_stack((r, fi))
print("Полярные координаты:")
print(polar_c)
x


Декартовы координаты:
[[56  2]
 [24 61]
 [22 31]
 [ 0 54]
 [41 27]
 [58 10]
 [19 60]
 [64 66]
 [ 3 38]
 [42 73]]
Полярные координаты:
[[5.60357029e+01 3.56991127e-02]
 [6.55515065e+01 1.19595561e+00]
 [3.80131556e+01 9.53604935e-01]
 [5.40000000e+01 1.57079633e+00]
 [4.90917508e+01 5.82352946e-01]
 [5.88557559e+01 1.70735211e-01]
 [6.29364759e+01 1.26412001e+00]
 [9.19347595e+01 8.00781565e-01]
 [3.81182371e+01 1.49201237e+00]
 [8.42199501e+01 1.04870471e+00]]


array([56, 24, 22,  0, 41, 58, 19, 64,  3, 42])

### Матричная алгебра

Перемножение массивов является поэлементным. Для матричного умножения необходимо использовать функцию `dot`:

In [144]:
m = np.array([[1., 2.], [3., 4.]])
m

array([[1., 2.],
       [3., 4.]])

In [145]:
m * m

array([[ 1.,  4.],
       [ 9., 16.]])

In [146]:
m.dot(m)

array([[ 7., 10.],
       [15., 22.]])

In [147]:
m @ m

array([[ 7., 10.],
       [15., 22.]])

### Трансформирование массивов

Для транспонирования матриц используется либо `.T`, либо функция `transpose`:

In [ ]:
m

In [ ]:
m.T

In [ ]:
m.transpose()

Другие математические функции:

In [148]:
c = np.matrix([[1j, 2j], [3j, 4j]])
c

matrix([[0.+1.j, 0.+2.j],
        [0.+3.j, 0.+4.j]])

In [149]:
np.conjugate(c)

matrix([[0.-1.j, 0.-2.j],
        [0.-3.j, 0.-4.j]])

Эрмитово-сопряженная матрица(transpose + conjugate):

In [150]:
c.H

matrix([[0.-1.j, 0.-3.j],
        [0.-2.j, 0.-4.j]])

Вещественная и мнимая части могут быть получены с помощью `real` и `imag`:

In [ ]:
np.real(c) # то же: c.real

In [ ]:
np.imag(c) # то же: c.imag

Модули элементов матрицы:

In [ ]:
np.abs(c)

### Упражнение 6
Найти ближайшее значение к числу 4 в векторе [4.3, 1.0, 3.9, 5.0, 2.0, 1.9]

In [ ]:
v = np.array([4.3, 1.0, 3.9, 5.0, 2.0, 1.9])
x = 4
i = np.abs(v - x).argmin()
otv = v[i]
print("Ближайшее значение к числу 4:", otv)


### Матричные вычисления

#### inverse

In [ ]:
np.linalg.inv(c) # то же: C.I 

In [ ]:
c.I * c

#### determinant

In [ ]:
np.linalg.det(c)

In [ ]:
np.linalg.det(c.I)

## Векторизация функций

Numpy предлагает средства для создания векторизованных оберток над функциями, которые изначально принимают на вход скалярные значения.

In [ ]:
def foo(x):
    if x >= 0:
        return 1
    else:
        return 0

In [ ]:
foo(np.array([-3, -2, -1, 0, 1, 2, 3]))

Эта функция работает для скалярных данных. 

Чтобы это функция принимала векторные значения, необходимо провести векторизацию с помощью функии `vectorize`:

In [ ]:
foo_vec = np.vectorize(foo)

In [ ]:
foo_vec(np.array([-3, -2, -1, 0, 1, 2, 3]))

# Matplotlib

* Пакет для Python, используемый для создания качественных 2D визуализацией (есть минимальная поддержка 3D)
* Возможность создавать интерактивные графики
* Добавление множества графиков на один рисунок с кастомным расположением
* Экспорт в различные форматы изображений
* Есть поддержка анимаций

In [ ]:
# Позволяет matplotlib отображать графики сразу в notebook.
%matplotlib inline

## Matplotlib API

Импортирование модуля `matplotlib.pyplot` под именем `plt`:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

Простейший пример построения графиков в matplotlib:

In [ ]:
x = np.linspace(-2., 2., 128, endpoint=True)
y1 = x**2
y2 = np.exp(x)
plt.plot(x, y1)
plt.plot(x, y2)
plt.xlabel(r'$x \in \mathbb{R}$', fontsize=12)
plt.ylabel(r'$y(x)$', fontsize=12)
plt.show()

Рекомендуется создавать отдельный объект для каждого более-менее сложного графика. Это можно реализоваться, например, с помощью функии `subplots`:

In [ ]:
# Сетка графиков -- 1x1. Размер задается с помощью figsize.
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 6))
ax.plot(x, y1, color="blue", linewidth=1.0, linestyle="-")
ax.plot(x, y2, color="green", linewidth=1.0, linestyle="--")
ax.grid()
ax.set_xlabel(r'$x$', fontsize=12)
ax.set_ylabel(r'$y$', fontsize=12)
plt.show()

Множественные графики

In [ ]:
# Создаем 2 графика (в 2 колонках)
fig, axes = plt.subplots(1, 2, figsize=(10, 6))
axes[0].plot(x, y1, 'r')
axes[1].plot(x, y2, 'b')
fig.tight_layout()

## Сохранение графиков

Текущий график можно сохранить, вызвав метод `savefig` класса `Figure`:

In [ ]:
fig.savefig("filename.png")

Также можно указать DPI и различные форматы:

In [ ]:
fig.savefig("filename.pdf", dpi=200)

### Легенды, описания осей и графиков

**Заголовок графика**


`axes.set_title("title")`

**Описания осей**


`axes.set_xlabel("x")
axes.set_ylabel("y")`

**Легенда**

Легенды могут создаваться двумя способами. Первый -- явно через метод `legend`:

`axes.legend(["curve1", "curve2"])`

Второй метод -- использование `label="label text"` при вызове `plot` с последующим вызовом метода `legend`: 

`axes.plot(x, x**2, label="curve1")
axes.plot(x, x**3, label="curve2")
axes.legend()`

Также можно выбрать расположение легенды на графике:

`ax.legend(loc=0) # автовыбор
ax.legend(loc='upper right')
ax.legend(loc='upper left')
ax.legend(loc='lower left')
ax.legend(loc='lower right')`

Пример использования описанного выше:

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, x**2, label="y = x**2")
ax.plot(x, x**3, label="y = x**3")
ax.legend(loc='upper left')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('title')